# Support Vector Machines

* A support vector machine (SVM) is a powerful and versatile machine learning model, capable of performing linear or nonlinear classification, regression, and even novelty detection.

* SVM is great for small to medium-sized nonlinear datasets (i.e., hundreds to thousands of instances), especially for classification tasks. However, they don't scale very well to very large datasets.

# A. Linear SVM Classification:

* Think of an SVM classifier as fitting the widest possible street between classes. (*Large Margin Classification*).
* SVMs are sensitive to the feature scales.


### A.1. Soft Margin Classification:

* Hard Margin Classification:
    * impose that all instances must be off the street and on the correct side.
    * 2 main issues w/ hard margin classification:
        * It only works if the data is linearly separable.
        * It is sensitive to outliers.

* Soft Margin Classification:
    * The objective is to find a good balance between keeping the street as large as possible and limiting the *margin violations* (i.e., instances that end up in the middle of the street or even on the wrong side).
